## Global variables

In [ ]:
suppliers = []
rfqs = []

## Classes definition

In [ ]:
from typing import Any


class Supplier:
    def __init__(self):
        self.µ_price_high_complexity = 0
        self.σ_price_high_complexity = 0
        self.µ_price_medium_complexity = 0
        self.σ_price_medium_complexity = 0
        self.µ_price_low_complexity = 0
        self.σ_price_low_complexity = 0

class Part_Number:
    def __init__(self, pn: str, complexity: str):
        self.pn = pn
        self.complexity = complexity

class RFQ:
    def __init__(self):
        pass